# Exponential growth

An interactive notebook for understanding exponential growth

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, save
from bokeh.io import curdoc, show
from bokeh.layouts import row, column, widgetbox, gridplot
from bokeh.models import ColumnDataSource, Legend, DatetimeTickFormatter
from bokeh.models.widgets import Slider, TextInput
from bokeh.io import output_notebook, output_file

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

Variables:

N0 = Initial population size

r = Discrete Growth rate

dt = Time step

Nt = Total number of steps (days)

max_Nt = Maximum number of steps (days) to be calculated

r0 = Basic reproduction number

In [2]:
N0 = 1
r = 1.3
dt = 1
Nt = 30
max_Nt = 40
r0 = 2

output_notebook()

# Create a datetime list
# today = datetime.datetime.today()
# today
# dt_list = [today - datetime.timedelta(days=x) for x in range(100)]
# dt_list

# dates = pd.date_range(pd.datetime.today(), periods=max_Nt + 2, freq = "1D")
# dates = dates.strftime('%B %d, %Y')
# dates

# dates = pd.date_range(pd.datetime.today(), periods=max_Nt + 2, freq = "1D")
# dates = dates.strftime('%B %d, %Y')

Loading BokehJS ...

In [3]:
# Numpy linspace takes arguments: start, stop, num
t = np.linspace(0, (max_Nt + 1)* dt, max_Nt + 2)
N = np.zeros(max_Nt + 2)
N_r0 = np.zeros(max_Nt + 2)

N[0] = N0
N_r0[0] = N0

for n in range (max_Nt + 1):
    N[n + 1] = N[n] + r*dt*N[n]
    N_r0[n + 1] = N_r0[n] * r0
    
y1 = N0 * np.exp(r*t)
y2 = N
y3 = N_r0

source = ColumnDataSource(data=dict(x=t, y1=y1, y2=y2, y3=y3))

p1 = figure(plot_height = 400, plot_width = 600, title = "Exponential Growth",
              tools = "crosshair,pan,reset,save,wheel_zoom",
              y_range = [0, 800])

p1.xaxis.axis_label = "Time (days)"
p1.yaxis.axis_label = "Number infected people"

res01 = p1.line('x', 'y1', source=source, line_width=3, line_alpha=0.6, color='blue')
res02 = p1.line('x', 'y2', source=source, line_width=3, line_alpha=0.6, color='red')
res03 = p1.line('x', 'y3', source=source, line_width=3, line_alpha=0.6, color='orange')

p2 = figure(plot_height = 400, plot_width = 400, title = "Exponential Growth Log Scale",
              tools = "crosshair,pan,reset,save,wheel_zoom", y_axis_type = "log")

p2.xaxis.axis_label = "Time (days)"
p2.yaxis.axis_label = "Number infected people (log scale)"

res01ex = p2.line('x', 'y1', source=source, line_width=3, line_alpha=0.6, color='blue')
res02ex = p2.line('x', 'y2', source=source, line_width=3, line_alpha=0.6, color='red')
res03ex = p2.line('x', 'y3', source=source, line_width=3, line_alpha=0.6, color='orange')

legend1 = Legend(items = [("Exp Growth Exact approach", [res01]), ("Exp Growth Numerical approach", [res02]), ("Exponential Growth r0", [res03])])
legend2 = Legend(items = [("Exp Growth Exact approach", [res01ex]), ("Exp Growth Numerical approach", [res02ex]), ("Exponential Growth r0", [res03ex])],
                                                                     location = (10,230))

p1.add_layout(legend1)
p2.add_layout(legend2)

# Set up widgets
N0_slider = Slider(title="Initial cases", value=N0, start=1, end=1000, step=1)
r_slider = Slider(title="Discrete Growth Rate", value=0, start=0, end=5, step=0.1)
r0_slider = Slider(title="Basic reproduction number", value=r0, start=0, end=5, step = 0.1)
Nt_slider = Slider(title="Total time (days)", value=Nt, start=10, end=max_Nt, step=1)
    
def update_data(attrname, old, new):
    # Get the current slider values
    a = N0_slider.value
    b = r_slider.value
    c = dt
    d = Nt_slider.value
    e = r0_slider.value
    
    D = np.zeros(d + 2)
    D[0] = a
    
    D_r0 = np.zeros(d + 2)
    D_r0[0] = a
    
    for n in range (d + 1):  
        D[n + 1] = D[n] + b*c*D[n]
        D_r0[n + 1] = D_r0[n] * e
        
    # Generate the new curve
    t1 = np.linspace(0, (d + 1)* dt, d + 2)
    y1 = a * np.exp(b*t1)
    y2 = D
    y3 = D_r0

    source.data = dict(x=t1, y1=y1, y2=y2, y3=y3)
    
    
for w in [N0_slider, r_slider, r0_slider, Nt_slider]:
    w.on_change('value', update_data)
    
    
inputs = widgetbox(N0_slider, r_slider, r0_slider, Nt_slider)
layout = row(p1, p2, widgetbox(N0_slider, r_slider, r0_slider, Nt_slider))
# layout = layout([[p1, inputs], [p2]], sizing_mode = "stretch_both")


def modify_doc(doc):
    doc.add_root(row(layout, width = 500))
    doc.title = "Sliders"
#     text.on_change('value', update_title)


handler = FunctionHandler(modify_doc)
app = Application(handler)
output_file("exponential_growth.html")
show(app)


